# PieceMatcher sample

Use the `PieceMatcher`


## Import


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from loguru import logger as lg
from rich import get_console
from rich import print as rprint
from rich.console import Console

# some magic to make rich work in jupyter
# https://github.com/Textualize/rich/issues/3483
# enable it for every cell output with %load_ext rich
console: Console = get_console()
console.is_jupyter = False

In [2]:
import itertools
from unittest.mock import MagicMock
from unittest.mock import patch

from pydantic import BaseModel

from snap_fit.config.aruco.aruco_board_config import ArucoBoardConfig
from snap_fit.config.aruco.aruco_detector_config import ArucoDetectorConfig
from snap_fit.config.types import EdgePos
from snap_fit.data_models.segment_id import SegmentId
from snap_fit.image.segment_matcher import SegmentMatcher
from snap_fit.params.snap_fit_params import get_snap_fit_params
from snap_fit.params.snap_fit_params import get_snap_fit_paths
from snap_fit.puzzle.sheet_aruco import SheetAruco
from snap_fit.puzzle.sheet_manager import SheetManager


## Params and config


In [ ]:
sf_paths = get_snap_fit_paths()
rprint(sf_paths)

### Load oca data


In [ ]:
# 1. Configure ArUco Board and Detector
# Using defaults which match the printed board used for 'data/oca'
board_config = ArucoBoardConfig(markers_x=5, markers_y=7)
detector_config = ArucoDetectorConfig(board=board_config)

# 2. Initialize SheetAruco helper
# crop_margin is automatically calculated from the detector configuration
sheet_aruco = SheetAruco(detector_config)

# 3. Define the loader function
# SheetAruco.load_sheet handles loading, rectification, and Sheet creation
aruco_loader = sheet_aruco.load_sheet

# 4. define base folder
paths = get_snap_fit_paths()
data_dir = paths.data_fol / "oca"
lg.info(f"Loading data from {data_dir}")

# 5. instantiate manager and load
manager = SheetManager()
manager.add_sheets(folder_path=data_dir, pattern="*.jpg", loader_func=aruco_loader)

## Piece match


In [ ]:
# change here